## Harmonicity Metrics Computation on MNE Epochs File

In this notebook, we demonstrate how to compute harmonicity metrics from an MNE epochs file using the Biotuner library. Harmonicity metrics provide insights into the periodic structure of EEG signals and can be useful for various applications in neuroscience research. Biotuner is a powerful tool for extracting these metrics by analyzing the harmonic architecture of M/EEG data.

In [7]:
# Import necessary modules
import mne
from mne import make_fixed_length_events
from biotuner.biotuner2d import biotuner_mne
from mne.io import concatenate_raws, read_raw_edf
from mne.datasets import eegbci

raw_fnames = eegbci.load_data(1, 1)
raws = [read_raw_edf(f, preload=True) for f in raw_fnames]
raw = concatenate_raws(raws)

# Define the biotuner parameters
bt_dict = {
    'peaks_function': 'fixed',
    'precision': 0.1,
    'fmin': 2,
    'fmax': 60,
    'n_peaks': 3,
    'harm_thresh': 30,
    'delta_lim': 150,
}


# Filter the data
raw.filter(l_freq=2., h_freq=60.)

# Get the names of the MEG channels
eeg_channels = mne.pick_types(raw.info, eeg=True)

# Keep only the first 10 EEG channels
eeg_channels = eeg_channels[:2]

# Update 'sf' in the bt_dict
bt_dict['sf'] = raw.info['sfreq']


# Epoch the data
tmin = 0.0  # start of each epoch (0s before the trigger)
tmax = 7.0  # end of each epoch (3s after the trigger)

# Create events
events = make_fixed_length_events(raw, start=0, stop=None, duration=tmax)

# Make sure only 3 epochs are created
if len(events) > 3:
    events = events[:3]

# Pick only the first 10 MEG channels
raw.pick_channels([raw.ch_names[i] for i in eeg_channels])

# Epoch the data
epochs = mne.Epochs(raw, events, tmin=tmin, tmax=tmax, baseline=None, preload=True)

# Compute biotuner metrics on epochs file
bt_df = biotuner_mne(epochs, bt_dict, savefile=True, savename='multimodal_raw')


[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
c:\Users\Antoine\anaconda3\envs\biotuner\Lib\site-packages\scipy\signal\_spectral_py.py:600: UserWarning: nperseg = 1600 is greater than input length  = 1121, using nperseg = 1121
  freqs, _, Pxy = _spectral_helper(x, y, fs, window, nperseg, noverlap,
c:\users\antoine\github\biotuner\biotuner\metrics.py:946: RuntimeWarning: divide by zero encountered in scalar divide
  harm_temp.append(1 / delta_norm)


In [8]:
bt_df

,trial,electrode,peaks_function,precision,fmin,fmax,n_peaks,harm_thresh,delta_lim,sf,...,sum_q_for_all_intervals,harm_sim,matrix_harm_sim,matrix_cons,matrix_denom,cons,tenney,harm_fit,harmsim,subharm_tension
0,0,0,fixed,0.1,2,60,5,30,150,160.0,...,1085327,8.92,2.889873,0.032337,1332.130159,0.096100,11.259724,4,8.915112,[0.20687287374966]
1,0,1,fixed,0.1,2,60,5,30,150,160.0,...,156833,9.01,2.896972,0.030319,507.058730,0.096353,10.512551,5,9.011115,[0.18720813295457783]
2,1,0,fixed,0.1,2,60,4,30,150,160.0,...,12228280,1.13,0.481869,0.004839,2772.647222,0.018917,14.211680,2,1.125127,[0.25226081305566683]
3,1,1,fixed,0.1,2,60,4,30,150,160.0,...,11929826,1.47,0.530435,0.005325,2275.941667,0.019180,14.039983,5,1.474938,[0.2175077245444831]
4,2,0,fixed,0.1,2,60,5,30,150,160.0,...,2718517,5.80,2.290880,0.024061,1115.077778,0.074855,10.494722,7,5.801397,[0.1968767555121499]
5,2,1,fixed,0.1,2,60,5,30,150,160.0,...,33486,7.98,2.853851,0.029816,291.877778,0.106638,8.833273,8,7.984592,[0.19687521906926572]
